In [ ]:
import cv2 as cv
import numpy as np
from keras.models import model_from_json
emotionDict={0:"Angry",1:"Disgusted",2:"Fearful",3:"Happy",4:"Neutral",5:"Sad",6:"Surprised"}
jsonFile=open('model\emotionModel.json','r')
loadedModelJson=jsonFile.read()
jsonFile.close()
emotionModel=model_from_json(loadedModelJson)
emotionModel.load_weights('model\emotionModel.h5')
print("Loaded model from disk")
# start webcam
# cap=cv.VideoCapture(0)
cap=cv.VideoCapture('sample2.mp4')
while True:
  ret,frame=cap.read()
  resizedImg=cv.resize(frame,(700,900))
  if not ret:
    break
  face_detector=cv.CascadeClassifier("haarcascades\haarcascade_frontalface_default.xml")
  gray_frame=cv.cvtColor(resizedImg,cv.COLOR_BGR2GRAY)
  # detect faces on camera
  numFace=face_detector.detectMultiScale(gray_frame,scaleFactor=1.3,minNeighbors=5)
  # take each face in camera and preprocess it
  for (x,y,w,h) in numFace:
    cv.rectangle(resizedImg,(x,y-50),(x+w,y+h+10),(0,255,0),4)
    roiGrayFrame=gray_frame[y:y+h,x:x+w]
    croppedImg=np.expand_dims(np.expand_dims(cv.resize(roiGrayFrame,(48,48)),-1),0)
    # predict emotions
    emotionPrediction=emotionModel.predict(croppedImg)
    maxIndex=int(np.argmax(emotionPrediction))
    cv.putText(resizedImg,emotionDict[maxIndex],(x+5,y-20),cv.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv.LINE_AA)
  cv.imshow('Sample',resizedImg)
  if cv.waitKey(1) & 0xFF==ord('q'):
    break
cap.release()
cv.destroyAllWindows()

Loaded model from disk
